In [2]:
# !wget https://huggingface.co/datasets/mesolitica/maksud-instructions/resolve/main/maksud.jsonl

In [3]:
import json
from glob import glob

In [4]:
files = sorted(glob('*-dictionary.jsonl'))
files

['negeri-johor-dictionary.jsonl',
 'negeri-kedah-dictionary.jsonl',
 'negeri-kelantan-dictionary.jsonl',
 'negeri-melaka-dictionary.jsonl',
 'negeri-pahang-dictionary.jsonl',
 'negeri-perak-dictionary.jsonl',
 'negeri-sabah-dictionary.jsonl',
 'negeri-sarawak-dictionary.jsonl',
 'negeri-sembilan-dictionary.jsonl',
 'negeri-terengganu-dictionary.jsonl']

In [5]:
from collections import defaultdict

languages = defaultdict(dict)
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            bahasa = f.split('/')[-1].split('-dictionary')[0].replace('-', ' ')
            perkataan = l['left'].lower()
            i = perkataan
            if l['dictionary']['maksud'][-1] == '.':
                maksud = l['dictionary']['maksud'][:-1]
            else:
                maksud = l['dictionary']['maksud']
            
            a = f"{maksud}. {l['dictionary']['penjelasan']}"
            languages[bahasa][i] = a

In [6]:
sublanguages = list(languages.keys())

In [7]:
baku = {}
with open('maksud.jsonl') as fopen:
    for l in fopen:
        try:
            l = json.loads(l)
            i = l['input'].split(': ')[1]
            if ':' in i:
                continue
            a = l['output'].split('Definisi : ')[1]
            if '\n(Kamus Dewan Edisi Keempat)' not in a:
                continue
            if len(a.replace('\n(Kamus Dewan Edisi Keempat)', '')) < 10:
                continue
                
            if len(a.replace('\n(Kamus Dewan Edisi Keempat)', '').split()) > 50:
                continue
                
            a = a.replace('\n(Kamus Dewan Edisi Keempat)', ' (berdasarkan Kamus Dewan Edisi Keempat)')
            baku[i] = a
        except:
            pass

len(baku)

43023

In [8]:
templates_q = [
    'apa mksud perkataan "{perkataan}" dalam dialek {bahasa}',
    'what is the meaning "{perkataan}" in dialect {bahasa}',
    'dlm dialek {bahasa}, apa maksud perkataan {perkataan}'
]

templates_a = [
    '"{perkataan}" bukanlah satu perkataan dari dialek {bahasa}, ia merupakan perkataan bahasa Melayu baku yang bermaksud {maksud}',
    "Ia bermaksud {maksud}, tetapi bukanlah satu perkataan dari dialek {bahasa}.",
]

In [11]:
import random

instructions_baku_negate = []

baku_keys = list(baku.keys())
for word in baku_keys:
    subs = random.sample(sublanguages, 5)
    count = 0
    for s in subs:
        if 'sembilan' in s:
            s_ = s.title()
        else:
            s_ = s.replace('negeri ', '').title()
        if word.lower() not in languages[s]:
            i = random.choice(templates_q).strip().format(perkataan = word, bahasa = s_)
            a = random.choice(templates_a).strip().format(perkataan = word, bahasa = s_, maksud = baku[word])
            instructions_baku_negate.append({
                'prompt_input': None,
                'input': i,
                'output': a,
            })
            
            count += 1
        
        if count >= 3:
            break
            
len(instructions_baku_negate)

129064

In [12]:
instructions_baku_negate[0]

{'prompt_input': None,
 'input': 'apa mksud perkataan "degung" dalam dialek Kedah',
 'output': '"degung" bukanlah satu perkataan dari dialek Kedah, ia merupakan perkataan bahasa Melayu baku yang bermaksud 1. bunyi (spt bunyi) gong; 2. (Sunda) sj gamelan. (berdasarkan Kamus Dewan Edisi Keempat)'}

In [14]:
sublanguages

['negeri johor',
 'negeri kedah',
 'negeri kelantan',
 'negeri melaka',
 'negeri pahang',
 'negeri perak',
 'negeri sabah',
 'negeri sarawak',
 'negeri sembilan',
 'negeri terengganu']

In [18]:
instructions = []

templates_q = [
    'apa mksud perkataan "{perkataan}" based on kamus dewan',
    'what is the meaning "{perkataan}" berdasarkan kamus dewan',
    'based on kamus dewan, apa maksud perkataan {perkataan}'
]

for word in baku_keys:
    alls = []
    for s in sublanguages:
        alls.append(word.lower() not in languages[s])
        
    if all(alls):
            
        i = random.choice(templates_q).strip().format(perkataan = word)
        a = baku[word]
        instructions.append({
            'prompt_input': None,
            'input': i,
            'output': a,
        })

In [19]:
len(instructions)

42098

In [24]:
import pandas as pd

df = pd.DataFrame(instructions_baku_negate + instructions)
df.head()

,prompt_input,input,output
0,None,"apa mksud perkataan ""degung"" dalam dialek Kedah","""degung"" bukanlah satu perkataan dari dialek K..."
1,None,"apa mksud perkataan ""degung"" dalam dialek Melaka","""degung"" bukanlah satu perkataan dari dialek M..."
2,None,"apa mksud perkataan ""degung"" dalam dialek Perak",Ia bermaksud 1. bunyi (spt bunyi) gong; 2. (Su...
3,None,"apa mksud perkataan ""berkolek"" dalam dialek Sabah","""berkolek"" bukanlah satu perkataan dari dialek..."
4,None,"what is the meaning ""berkolek"" in dialect Kela...","""berkolek"" bukanlah satu perkataan dari dialek..."


In [25]:
df.to_parquet('prepared-maksud-instructions.parquet')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-maksud-instructions.parquet",
    path_in_repo='data/maksud_instructions-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)